# 오피넷 상단메뉴로 들어가기

In [34]:
# import
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from glob import glob
from bs4 import BeautifulSoup
# import url
from urllib.parse import urlparse
import requests
# 카카오 키 불러오기
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [4]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.get('http://www.opinet.co.kr/user/main/mainView.do')
time.sleep(1)
driver.find_element_by_class_name('gnbTopa').click()
driver.find_element_by_xpath('//*[@id="header"]/div/ul/li[1]/ul/li[1]/a').click()
time.sleep(1)


In [23]:
gu =[]
for i in range(len(driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]').find_elements_by_tag_name('option'))):
    gu.append(driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]').find_elements_by_tag_name('option')[i].text)


In [25]:
gu

['시/군/구',
 '강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [26]:
del gu[0]

In [29]:
o_name = [] ; o_price =[]; o_addr =[] ; s = list(range(0,20000,2))
for search in gu:
    region = driver.find_element_by_id('SIGUNGU_NM0')
    region.send_keys(search)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="searRgSelect"]').click()
    time.sleep(1)
    for i in range(len(driver.find_elements_by_class_name('rlist'))):
        if len(driver.find_elements_by_class_name('rlist')[i].text) == 0 :
            break
        driver.find_elements_by_class_name('rlist')[i].find_element_by_tag_name('a').click()
        time.sleep(1)
        html = driver.page_source # 페이지의 elements모두 가져오기
        soup = BeautifulSoup(html, 'html.parser') 
        o_name.append(soup.find('div', 'header').get_text())
        o_addr.append(soup.find('label', {'id' : 'rd_addr'}).get_text())
        o_price.append(driver.find_elements_by_class_name('price')[s[i]].text)

In [180]:
c_df = pd.DataFrame({
    '상호' : o_name,
    '가격' : o_price,
    '주소' : o_addr
})

In [181]:
len(o_name),len(o_price),len(o_addr)

(488, 488, 488)

### 주소가 잘못된 것들이 몇몇개가 나와서 검색어로 찾았더니 더 찾기 힘들어짐

In [182]:
c_df = c_df.sort_values(by = '가격').reset_index(drop=True)
c_df['주소'][202] = '서울 송파구 위례서로 260'
c_df['주소'][290] = '서울 마포구 대흥로 60'
c_df['주소'][441] = '서울 영등포구 국회대로 665'

In [183]:
lat_list =[] ; lng_list = [] 
for i in range(len(c_df)):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + c_df['주소'][i]
    result = requests.get(urlparse(url).geturl(),headers={"Authorization": "KakaoAK "+kmap_key}).json()
    lat = float(result['documents'][0]['address']['y'])
    lng = float(result['documents'][0]['address']['x'])
    lat_list.append(lat)
    lng_list.append(lng)

c_df['lat'] = lat_list
c_df['lng'] = lng_list

In [186]:
colors = ['green', 'red','black']

### 서울의 모든 주유소 

In [195]:
import folium
map = folium.Map(location=[c_df['lat'].mean(), c_df['lng'].mean()], zoom_start=10)

for n in range(len(c_df)) :
    if n <= 10:
        k = 1
    elif n >= len(c_df) - 10:
        k = 2 
    else:
        k = 0
    folium.Marker([c_df['lat'][n], 
    c_df['lng'][n]],popup=c_df['상호'][n],
    tooltip=f'{c_df["가격"][n]}', icon=folium.Icon(color=colors[k])).add_to(map)

map